#### *IBM Data Science Certification Capstone Project - authored by Julien Girault, data scientist scholar*
# Find your Favourite Tube Station before Moving to London!

# Project introduction

Many French people move from Paris to London to improve their English and live a new experience there.

But what is the best place they should choose for settling down?

When moving to a new city, one will likely choose their new address based on public transports: they will search the lines that can bring them quickly and directly from their new home to their office.

__"London tube" and "Paris métro" public transports are comparably big with approximately 400 stations__... So here is the question that comes next: how can one know the kind of surroundings they are likely to find around each station? How can they decide where to settle down? Which station should they prefer with so many possibilities?

This project gives French newcomers the opportunity to find the list of tube stations in London that best fit their taste, based on a comparison with something they already know: Paris's metro stations!

For each London tube station, we expose a list of Paris stations that are similar. Here is a full report describing how this is done.

You can also find this full report and corresponding source code in [my jupyter notebook of the project on my github account here](https://github.com/juliengirault52/Coursera_Capstone/blob/main/capstone_project.ipynb).


# Data Description

There are two data entities for this project, here is a short description for both of them to help the reader have a quick understanding:
### Stations coordinates
For this project, I downloaded the transports stations coordinates released by the public services for both London [1] and Paris [2].

### List of venues available around each stations
To compare the stations, I used the data available via the Foursquare API for each station coordinates, exploring the list of venues reachable within a five-minute walk (300 meters).

# Methodology

The main idea of the project was __to mix all stations of both London and Paris cities and cluster them__. The comparision of the stations had to be based only on the services offered in their surroundings (i.e. without taking into account the location). And finally when able to compare, create groups of stations that are similar.

### Why need an AI algorithm?
For a two-dimension problem, when one wants to create clusters, this can be done simply by visualizing the data and there is no need to use AI. But in our case, our dataset had about 400 features, which means about 400 dimensions! Even after trying simplifying the data, which will be covered later in the report, there still was more than 200 features. This is why we needed to use a clustering algorithm to reach the objective.

### London-Paris "city-balance": an adhoc quality score
One of the risks was that the clusters would regroup stations only from the same city (Paris or London). To make sure that the resulting clusters would regroup a balanced quantity of stations from each city, I computed a "balance score". The score must be as low as possible. It is the distance to 50% for the ratio "number of stations of Paris" / "total number of stations" (*score=abs(0.50-np/nl)*).
ex:
```
Cluster Labels-3 cities-balance score: 0.3574861584519108 inertia: 89.97750663333983
Cluster Labels-4 cities-balance score: 0.34106974568562276 inertia: 86.00200658957111
Cluster Labels-5 cities-balance score: 0.34197384655531804 inertia: 82.58403862899996
Cluster Labels-6 cities-balance score: 0.37053549083395193 inertia: 79.11207334558907
Cluster Labels-7 cities-balance score: 0.37329728602870704 inertia: 77.13651635333981
```
### Data preparation
#### Categories comparison
A manual job was useful to increase the matching of venues categories between Paris and London. For instance, I needed to transform "Pizza Place" (which figured only in London) to "Italian Restaurant", or there was also very specific data such as "Auvergne Restaurant" that became "Restaurant". And of course the English "Pub" and French "Bistro" needed to be turned to a more generic naming: "Bar".

To do so, I created a "categories_to_merge.csv" with an ordered list of the most common categories and identified which ones were unbalanced between London and Paris.
#### Data simplification
To reduce features, I kept only the categories that were in the N-top categories. This allowed having about 250 features instead of 402.

### why use the k-means algorithm? (iterating and selecting the best score)
One of the "k-means clustering" algorithm advantages is that one can chose how many clusters will be created at the beginning of the process. Using the "k-means clustering" algorithm was a good way to iterate in order to get the best "city-balance" quality score. As a matter of facts, in that algorithm we know that the highest number of clusters you create, the more similar the stations will be. But on the other hand, we wished to get the best balance as possible so I chose to run multiple times the algorithm and save the best balance result. This was a good way to get the perfect equilibrium between similarity and "city balance".

# Results
Here are the two maps generated by the clustering process.

# Discussion
Although we used k-means for its advantage explained above, it could be interesting to run a DBSCAN algorithm to see how many clusters it would compute. DBSCAN does not require to specify the number of clusters and find the best fitting number. After this is done, it would also be interesting to see what London-Paris "city-balance" score this would give.

# Conclusion
It is amasing how clustering can help to distinguish clusters when data is very complex. Now let's hope that this London tube list will help people who move to find the place where they will settle down!


## References

[1] [London Tube Stations List (CSV)](https://www.whatdotheyknow.com/request/512947/response/1238210/attach/3/Stations%2020180921.csv.txt?cookie_passthrough=1),

[2] [Paris Transports Stations List website](https://data.iledefrance-mobilites.fr/explore/dataset/emplacement-des-gares-idf/download/?format=json&refine.mode=Metro&timezone=Europe/Berlin&lang=fr)

[3] [Coursera Project Training Exercise "The Battle of Neighborhoods"](https://www.coursera.org/learn/applied-data-science-capstone)

In [1]:
import pandas as pd
import io
import requests

#London tube stations coordinates:
req=requests.get("https://www.whatdotheyknow.com/request/512947/response/1238210/attach/3/Stations%2020180921.csv.txt?cookie_passthrough=1").content
df_london=pd.read_csv(io.StringIO(req.decode('utf-8')))

#Paris metro stations coordinates:
req=requests.get("https://data.iledefrance-mobilites.fr/explore/dataset/emplacement-des-gares-idf/download/?format=json&refine.mode=Metro&timezone=Europe/Berlin&lang=fr").content
df_paris=pd.read_json(io.StringIO(req.decode('utf-8')))

In [2]:
df_paris.head(4)

,datasetid,recordid,fields,geometry,record_timestamp
0,emplacement-des-gares-idf,723289fe50c959f7e63d75b17870762aa8eaddd4,"{'res_stif': 110.0, 'cod_ligf': 14.0, 'nom_iv'...","{'type': 'Point', 'coordinates': [2.3891158073...",2020-01-15T11:22:48.576+01:00
1,emplacement-des-gares-idf,2f98f2e1ee73e414cf64bae428caa96ba114be23,"{'res_stif': 110.0, 'cod_ligf': 4.0, 'nom_iv':...","{'type': 'Point', 'coordinates': [2.3209981919...",2020-01-15T11:22:48.576+01:00
2,emplacement-des-gares-idf,dafc950d65ec51317aa65aaba7a12fb5a0cfc396,"{'res_stif': 110.0, 'cod_ligf': 15.0, 'nom_iv'...","{'type': 'Point', 'coordinates': [2.2781616712...",2020-01-15T11:22:48.576+01:00
3,emplacement-des-gares-idf,5bc1c5091428bb56801455343b0cd58fca8d4179,"{'res_stif': 110.0, 'cod_ligf': 1.0, 'nom_iv':...","{'type': 'Point', 'coordinates': [2.3693205849...",2020-01-15T11:22:48.576+01:00


In [3]:
df_london.head(2)

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114


Need to fetch the coordinates into the table dictionary structure the are stored in, easy with pandas!

In [4]:
for i, station in enumerate(df_paris["fields"]):
    df_paris["fields"][i]=station["nom_iv"]
for i, geom in enumerate(df_paris["geometry"]):
    df_paris["geometry"][i]=geom["coordinates"][0]
    df_paris["record_timestamp"][i]=geom["coordinates"][1]

Let's regroup the stations in a common table with correct column names

In [5]:
df_paris.drop({"datasetid"}, 1, inplace=True)
df_paris.rename(columns={"recordid":"City","fields": "Station", "geometry": "Longitude","record_timestamp": "Latitude"}, inplace=True)
df_london.rename(columns={"FID": "City","x": "Longitude", "y": "Latitude", "NAME": "Station"}, inplace=True)
df_london.drop({'NORTHING', 'Zone', 'LINES', 'OBJECTID', 'EASTING', 'NETWORK'}, 1, inplace=True)
df_london["City"]="London"
df_paris["City"]="Paris"
df=df_london.append(df_paris[df_london.columns], ignore_index=True)

So here what the merged data source look like:

In [6]:
df

,City,Station,Longitude,Latitude
0,London,Temple,-0.112644,51.5105
1,London,Blackfriars,-0.10202,51.5111
2,London,Mansion House,-0.0924953,51.5113
3,London,Cannon Street,-0.088801,51.511
4,London,Monument,-0.0845023,51.5102
...,...,...,...,...
857,Paris,Voltaire,2.38072,48.8575
858,Paris,Wagram,2.30467,48.8838
859,Paris,Saint-Lazare,2.32426,48.8758
860,Paris,Trinité d'Estienne d'Orves,2.33254,48.8763


If all the stations had distinct names we could have used only "Stations" as a primary key for the project, but there is one...

In [7]:
#check how many stations in Paris and London have the same name:
res=df_paris.merge(df_london, on="Station", how="inner")
res

,City_x,Station,Longitude_x,Latitude_x,City_y,Longitude_y,Latitude_y
0,Paris,Temple,2.36154,48.8667,London,-0.112644,51.510474


Now that we have the stations, we can load Foursquare and init getNearbyVenues function (borrowed from previous lab) and get top venues for each Station

In [8]:
f=open("/resources/IBM Capstone Project/Coursera_Capstone/credentials.txt","r")
lines=f.readlines()
API_id=lines[4][:-1]
API_secret=lines[7][:-1]
f.close()
 

CLIENT_ID = API_id
CLIENT_SECRET = API_secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500

def getNearbyVenues(stations_done_list, venues_list, names, cities, latitudes, longitudes, radius=300):
    
    print("Fetching NearbyVenues started!")
    for name, city, lat, lng in zip(names, cities, latitudes, longitudes):
###        print(name)
        if (city+name) not in stations_done_list:
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
                
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            # return only relevant information for each nearby venue
            venues_list.append([(
                city, 
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
            stations_done_list.append(city+name)        
    print("NearbyVenues are ready!")


    

<a id='item2'></a>


In [ ]:
if 'venues_list' not in globals():
    venues_list=[]
    stations_done_list=[]
#force init:
#venues_list=[]
#stations_done_list=[]

In [13]:
#if you get an error, you can repeat until the end is reached without error, the venues_list is saved and the data can be loaded with multiple executions.
getNearbyVenues(stations_done_list, venues_list,names=df['Station'],cities=df['City'], 
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'], radius=300
                                  )

Fetching NearbyVenues started!
NearbyVenues are ready!


In [14]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['City', 'Station',
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [15]:
nearby_venues.head(100)

,City,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,London,Temple,51.510474,-0.112644,Two Temple Place,51.511523,-0.112236,History Museum
1,London,Temple,51.510474,-0.112644,Temple Gardens,51.511154,-0.111472,Park
2,London,Temple,51.510474,-0.112644,The Southbank Observation Point,51.508297,-0.111180,Scenic Lookout
3,London,Temple,51.510474,-0.112644,HQS Wellington,51.510679,-0.112214,Boat or Ferry
4,London,Temple,51.510474,-0.112644,The Queen's Walk,51.508308,-0.110853,Scenic Lookout
...,...,...,...,...,...,...,...,...
95,London,Tower Hill,51.509434,-0.074914,Virgin Active,51.511804,-0.074589,Gym / Fitness Center
96,London,Aldgate,51.513982,-0.074236,Hotel Indigo,51.512740,-0.075920,Hotel
97,London,Aldgate,51.513982,-0.074236,Dorsett City London,51.514036,-0.075812,Hotel
98,London,Aldgate,51.513982,-0.074236,Treves & Hyde,51.514114,-0.070606,Restaurant


We now have a list of 17k venues!


In [16]:
print(nearby_venues.shape)

(17309, 8)


# Data preparation

In [19]:

pd.set_option('display.max_rows', 100)
nearby_venues.loc[nearby_venues["Venue Category"]=="French Restaurant", "Venue Category"]="Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Brasserie", "Venue Category"]="Gastropub"
nearby_venues.loc[nearby_venues["Venue Category"]=="Gym / Fitness Center", "Venue Category"]="Gym"
nearby_venues.loc[nearby_venues["Venue Category"]=="Pub", "Venue Category"]="Bar"
nearby_venues.loc[nearby_venues["Venue Category"]=="Wine Bar", "Venue Category"]="Bar"
nearby_venues.loc[nearby_venues["Venue Category"]=="Bistro", "Venue Category"]="Bar"
nearby_venues.loc[nearby_venues["Venue Category"]=="Cocktail Bar", "Venue Category"]="Bar"
nearby_venues.loc[nearby_venues["Venue Category"]=="Coffee Shop", "Venue Category"]="Café"
nearby_venues.loc[nearby_venues["Venue Category"]=="Grocery Store", "Venue Category"]="Supermarket"
nearby_venues.loc[nearby_venues["Venue Category"]=="Pizza Place", "Venue Category"]="Italian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Sushi Restaurant", "Venue Category"]="Japanese Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Creperie", "Venue Category"]="Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Chinese Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Vietnamese Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Thai Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Korean Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Cambodian Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Dim Sum Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Cantonese Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Szechuan Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Ramen Restaurant", "Venue Category"]="Asian Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Lebanese Restaurant", "Venue Category"]="Mediterranean Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Israeli Restaurant", "Venue Category"]="Mediterranean Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Corsican Restaurant", "Venue Category"]="Mediterranean Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Falafel Restaurant", "Venue Category"]="Mediterranean Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Greek Restaurant", "Venue Category"]="Mediterranean Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Turkish Restaurant", "Venue Category"]="Middle Eastern Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Moroccan Restaurant", "Venue Category"]="Middle Eastern Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Kebab Restaurant", "Venue Category"]="Middle Eastern Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Persian Restaurant", "Venue Category"]="Middle Eastern Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="English Restaurant", "Venue Category"]="Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Auvergne Restaurant", "Venue Category"]="Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Alsatian Restaurant", "Venue Category"]="Restaurant"
nearby_venues.loc[nearby_venues["Venue Category"]=="Tapas Restaurant", "Venue Category"]="Spanish Restaurant"
nearby_venues=nearby_venues[~nearby_venues["Venue Category"].str.contains('Station')] #drops bus, metro and gaz stations
nearby_venues=nearby_venues[nearby_venues["Venue Category"]!="Platform"]
nearby_venues=nearby_venues[nearby_venues["Venue Category"]!="Bus Stop"]


nearby_venues.groupby(['Venue Category', 'City']).count().sort_values("Station", ascending=False).head(100)
#nearby_venues.groupby(['Venue Category', 'City']).count().sort_values("Station", ascending=False).to_csv("categories_to_merge")




Station  Station Latitude  \
Venue Category                City                                
Restaurant                    Paris      1280              1280   
Café                          London     1081              1081   
Bar                           London      808               808   
                              Paris       728               728   
Hotel                         Paris       664               664   
Italian Restaurant            Paris       464               464   
Asian Restaurant              Paris       412               412   
Italian Restaurant            London      409               409   
Supermarket                   London      392               392   
Café                          Paris       352               352   
Hotel                         London      319               319   
Asian Restaurant              London      314               314   
Japanese Restaurant           Paris       309               309   
Bakery                        Paris       258               258   
Restaurant                    London      253               253   
Supermarket                   Paris       220               220   
Gym                           London      214               214   
Sandwich Place                London      209               209   
Indian Restaurant             London      180               180   
Bakery                        London      177               177   
Plaza                         Paris       171               171   
Middle Eastern Restaurant     London      168               168   
Clothing Store                London      155               155   
Japanese Restaurant           London      150               150   
Fast Food Restaurant          London      141               141   
Burger Joint                  London      128               128   
Pharmacy                      London      117               117   
Park                          London      116               116   
Mediterranean Restaurant      Paris       108               108   
Sandwich Place                Paris       106               106   
Mediterranean Restaurant      London      104               104   
Middle Eastern Restaurant     Paris        92                92   
Gym                           Paris        88                88   
Bookstore                     London       79                79   
Theater                       Paris        78                78   
Gastropub                     Paris        72                72   
Burger Joint                  Paris        68                68   
Bookstore                     Paris        67                67   
Clothing Store                Paris        62                62   
Seafood Restaurant            Paris        61                61   
Portuguese Restaurant         London       60                60   
Theater                       London       58                58   
Tea Room                      Paris        56                56   
Breakfast Spot                London       55                55   
Plaza                         London       55                55   
Convenience Store             London       54                54   
Indian Restaurant             Paris        54                54   
Park                          Paris        54                54   
Cosmetics Shop                Paris        53                53   
Deli / Bodega                 London       49                49   
Dessert Shop                  Paris        47                47   
Garden                        London       47                47   
Wine Shop                     Paris        46                46   
Fish & Chips Shop             London       46                46   
Spanish Restaurant            London       46                46   
Ice Cream Shop                Paris        43                43   
Art Gallery                   Paris        43                43   
Chocolate Shop                Paris        43                43   
Pastry Shop       

In [20]:
venues=nearby_venues
venues.groupby(['Station','City']).count().head()

,,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,City,,,,,,
ABBEY ROAD - DLR,London,1,1,1,1,1,1
ALL SAINTS - DLR,London,15,15,15,15,15,15
Abbesses,Paris,70,70,70,70,70,70
Acton Central,London,5,5,5,5,5,5
Acton Town,London,12,12,12,12,12,12


In [31]:
len(venues['Station'].unique())

752

In [32]:
# vs

len(df['Station'])

862

<p style="color:red;">IMPORTANT : we notice here that 111 Stations returned zero venues we will need to add manually a cluster for "no venue" Stations</p>

<a id='item3'></a>


Analyze the typology of venues found around each Station, creating onehot encoding

In [96]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# one hot encoding
stations_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
# add Station column back to dataframe
stations_onehot['City'] = venues['City'] 
stations_onehot['Station'] = venues['Station'] 

# move Station column to the first column
fixed_columns = [stations_onehot.columns[-1]] + list(stations_onehot.columns[:-1])
stations_onehot = stations_onehot[fixed_columns]
fixed_columns = [stations_onehot.columns[-1]] + list(stations_onehot.columns[:-1])
stations_onehot = stations_onehot[fixed_columns]

#stations_onehot.head()
#### Next, let's group rows by Station+City and by taking the mean of the frequency of occurrence of each category
stations_grouped = stations_onehot.groupby(['Station','City']).mean().reset_index()
print(stations_onehot.shape)
stations_grouped.head(3)

(16583, 402)


,Station,City,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basque Restaurant,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Ch'ti Restaurant,Champagne Bar,Cheese Shop,Chocolate Shop,Church,Cigkofte Place,Circus,Climbing Gym,Clothing Store,College Cafeteria,College Gym,College Residence Hall,College Science Building,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Gym,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Lyonese Bouchon,Mac & Cheese Joint,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office

In [97]:
from statistics import mean
from sklearn.cluster import KMeans

import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station','City']

num_top_venues=5
kclusters_max = 20
bestScore=1
#for num_top_venues in range(3,num_top_venues_max+1)

#First, let's write a function to sort the venues in descending order.
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Stations_venues_sorted = pd.DataFrame(columns=columns)
Stations_venues_sorted['Station'] = stations_grouped['Station']
Stations_venues_sorted['City'] = stations_grouped['City']


for ind in np.arange(stations_grouped.shape[0]):
    Stations_venues_sorted.iloc[ind, 2:] = return_most_common_venues(stations_grouped.iloc[ind, :], num_top_venues) 
    
    
Stations_venues_sorted.head(10)
#Keep only top common venues for k mean computing:

# remove venue category if not in top
print(stations_grouped.shape)
stations_grouped_idexRef=stations_grouped.copy()
isIn=stations_grouped.columns.isin(Stations_venues_sorted.iloc[:,-1])+stations_grouped.columns.isin(Stations_venues_sorted.iloc[:,-2])+stations_grouped.columns.isin(Stations_venues_sorted.iloc[:,-3])+stations_grouped.columns.isin(Stations_venues_sorted.iloc[:,-4])+stations_grouped.columns.isin(Stations_venues_sorted.iloc[:,-5])
for col_index, isCommonTop in enumerate(isIn):
#    print(stations_grouped_idexRef.columns[col_index])
    if not isCommonTop and col_index>1:
        stations_grouped.drop(stations_grouped_idexRef.columns[col_index], axis=1, inplace=True)
###        print(stations_grouped_idexRef.columns[col_index]+" was dropped")
print(stations_grouped.shape)


### 4. Cluster Stations
#### Run _k_-means to cluster the Station into X clusters.
for kclusters in range (3,kclusters_max+1):

    stations_grouped_clustering = stations_grouped.drop(['Station','City'], 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=num_top_venues*kclusters).fit(stations_grouped_clustering)

    # add clustering labels
    Stations_venues_sorted.insert(0, 'Cluster Labels'+'-'+str(kclusters), kmeans.labels_)

    #Start evaluate balance Paris / Londres pour chaque cluster. Le faire dans la version top=x et K=Y puis faire évoluer.

    #récolter le score d'equilibire moyen : moyenne des scores pour ce K-N
    #for each cluster
    clustScores=[]
    for clust in Stations_venues_sorted["Cluster Labels"+'-'+str(kclusters)].unique():
    #    compute sum of Paris / total and calculate distance to "50%" which is the best score possible
        clustScores.append(abs(0.50-Stations_venues_sorted[Stations_venues_sorted["Cluster Labels"+'-'+str(kclusters)]==clust].loc[Stations_venues_sorted["City"]=="Paris"].size /
                               Stations_venues_sorted[Stations_venues_sorted["Cluster Labels"+'-'+str(kclusters)]==clust].size))

    #on copie ce K-N s'il est plus proche de 50% que celui qui était enregistré
    print("Cluster Labels"+'-'+str(kclusters) + " cities-balance score: " + str(mean(clustScores)) + " inertia: "+ str(kmeans.inertia_))
    if mean(clustScores)<bestScore:
        bestScore=mean(clustScores)
        best_clustScores=clustScores
        bestLabel="Cluster Labels"+'-'+str(kclusters)
        bestStations_venues_sorted=Stations_venues_sorted

(753, 402)
(753, 252)
Cluster Labels-3 cities-balance score: 0.3505701264019547 inertia: 95.07483342129092
Cluster Labels-4 cities-balance score: 0.3392919090215728 inertia: 91.20580271148025
Cluster Labels-5 cities-balance score: 0.36868131445282437 inertia: 87.44226993821589
Cluster Labels-6 cities-balance score: 0.3614490804011694 inertia: 84.26123399222773
Cluster Labels-7 cities-balance score: 0.37818442591770385 inertia: 82.26891204024325
Cluster Labels-8 cities-balance score: 0.3613826476159875 inertia: 80.80129257865467
Cluster Labels-9 cities-balance score: 0.3801164324931853 inertia: 79.04675951388617
Cluster Labels-10 cities-balance score: 0.3545257600399716 inertia: 77.24193989899149
Cluster Labels-11 cities-balance score: 0.3667837914023669 inertia: 76.27781234250926
Cluster Labels-12 cities-balance score: 0.3654054642166215 inertia: 75.16163624217525
Cluster Labels-13 cities-balance score: 0.3939902820154222 inertia: 74.47725689696492
Cluster Labels-14 cities-balance scor

In [ ]:
top=5 + cut
Cluster Labels-14 cities-balance score: 0.33960713148509347 inertia: 73.12300157030559

#Cluster Labels-10 0.29785175311725703 num_venues : 6
#Cluster Labels-11 0.30647333010757244 num_venues : 4

print(bestLabel)
#bestStations_venues_sorted
#best_clustScores



In [116]:
Stations_venues_sorted.drop("Cluster Labels-3", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-4", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-5", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-6", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-7", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-8", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-9", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-10", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-11", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-12", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-13", 1,inplace=True)
#Stations_venues_sorted.drop("Cluster Labels-14", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-15", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-16", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-17", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-18", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-19", 1,inplace=True)
Stations_venues_sorted.drop("Cluster Labels-20", 1,inplace=True)
#e

In [123]:
#Stations_venues_sorted.head(500).sort_values(["Cluster Labels-14","1st Most Common Venue","2nd Most Common Venue"])

Stations_venues_sorted[Stations_venues_sorted["Cluster Labels-14"]==0].iloc[:,-1]

18                        Bakery
21                        Bakery
26                    Food Truck
56                        Lounge
69                           Bar
70      Mediterranean Restaurant
75                  Dance Studio
76            Italian Restaurant
96                         Plaza
98                         Hotel
133                    Nightclub
167             Asian Restaurant
185          Empanada Restaurant
198                  Cheese Shop
229           Italian Restaurant
247               Breakfast Spot
254               Cosmetics Shop
258                         Café
286     Mediterranean Restaurant
301          Empanada Restaurant
348                   Steakhouse
358                          Bar
360                        Plaza
364                          Bar
382             Asian Restaurant
394               Sandwich Place
396                       Bakery
398                    Gift Shop
422    Middle Eastern Restaurant
426     Bike Rental / Bike Share
427       

In [ ]:
#stations_merged[stations_merged['Cluster Labels']==np.nan].head()
# merge stations_grouped with stations_data to add latitude/longitude for each Station
stations_merged = df.join(Stations_venues_sorted.set_index(['Station','City']), on=['Station','City'])
stations_merged.head()

When we added Station without venues, we introduced some cells with class "nan", let's create one last group so now we have kclusters+1 clusters.

In [ ]:
stations_merged.replace({'Cluster Labels': np.nan},kclusters, inplace=True)
stations_merged=stations_merged.astype({'Cluster Labels': 'int32'}, copy=False)
kclusters=kclusters+1

### 5. Examine Clusters and add explicit names



In [ ]:
stations_merged['Cluster explicit']=""

Cluster 0 seems to be "Cafes & restaurants district"


In [ ]:
stations_merged.loc[stations_merged['Cluster Labels'] == 0, ['Cluster explicit']] = "Cafes & restaurants district"

stations_merged.loc[stations_merged['Cluster Labels'] == 0, stations_merged.columns[[1] + list(range(5, stations_merged.shape[1]))]].head(6)


Cluster 1 seems to be "suburban district" with big facilities like Distribution centers or baseball fields


In [ ]:
stations_merged.loc[stations_merged['Cluster Labels'] == 1, ['Cluster explicit']] = "suburban district"

stations_merged.loc[stations_merged['Cluster Labels'] == 1, stations_merged.columns[[1] + list(range(5, stations_merged.shape[1]))]].head(6)

Cluster 2 seems to be "Fastfood and distribution district"


In [ ]:
stations_merged.loc[stations_merged['Cluster Labels'] == 2, ['Cluster explicit']] = "Fastfood and distribution district"

stations_merged.loc[stations_merged['Cluster Labels'] == 2, stations_merged.columns[[1] + list(range(5, stations_merged.shape[1]))]].head(6)

Cluster 3 seems to be "district with parks"


In [ ]:
stations_merged.loc[stations_merged['Cluster Labels'] == 3, ['Cluster explicit']] = "Parks district"

stations_merged.loc[stations_merged['Cluster Labels'] == 3, stations_merged.columns[[1] + list(range(5, stations_merged.shape[1]))]].head(6)

Cluster 4 was added because there are no venues so let's call it "no venues district"

In [ ]:
stations_merged.loc[stations_merged['Cluster Labels'] == 4, ['Cluster explicit']] = "no venues district"

stations_merged.loc[stations_merged['Cluster Labels'] == 4, stations_merged.columns[[1] + list(range(5, stations_merged.shape[1]))]].head(6)

Finally, let's visualize the resulting clusters


In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if needed
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map zoomed on Toronto
map_clusters = folium.Map(location=[43.7, -79.3831843], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, expl in zip(stations_merged['Latitude'], stations_merged['Longitude'], stations_merged['Station'], stations_merged['Cluster Labels'], stations_merged['Cluster explicit']):
    label = folium.Popup(str(poi) + " - " + expl, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters